Prototype where I write down all steps in case this becomes an instructional blog later.

Approach:
1. Set up connection to Strava
2. Set up local pgres database
3. Design
4. Retrieve data using Strava API
5. Load data to database

# Export strava data (using Stravalib) to local

This notebook includes the code as dicussed in the following Medium blog post:
https://medium.com/@mandieq/accessing-user-data-via-the-strava-api-using-stravalib-d5bee7fdde17

Documentation: authentication
https://stravalib.readthedocs.io/en/latest/get-started/authenticate-with-strava.html

Documentation: working with activities
https://stravalib.readthedocs.io/en/latest/get-started/activities.html



## Connection to Strava

### Create a new application
Log into strava.com, navigate to your profile, Settings, My API Application. Create an application.
https://www.strava.com/settings/api

Create a new application. Callback domain = localhost. Other parameters are arbitrary. Upload an icon. 

Make note of the client id and client secret. The other 2x tokens aren't useful because the scope isn't enough to retrieve data.


Load ID and secret for your Strava App as set up via Strava's developer area:

In [1]:
import json
from stravalib import Client
client = Client()

In [2]:
CLIENT_ID = '108084'
CLIENT_SECRET = 'bd51d6ce5c6c2e663f195859b3b092631bcd6242'
print ('client_ID=', CLIENT_ID, 'secret=', CLIENT_SECRET)

client_ID= 108084 secret= bd51d6ce5c6c2e663f195859b3b092631bcd6242


### One time authentication with athlete

Only needs to be done once in order to get auth token. Hereafter you use the refresh token to get a new one when the old one runs out. 

The cells below need to be converted back to code cells in order to run them. Note: **once only**!

Copy/paste the resulting URL into a browser. Click the Authorize button. Copy paste the resulting URL.
Copy down the code in the middle of the URL (between code= and scope=)


In [6]:
url = client.authorization_url(client_id=CLIENT_ID,
                               redirect_uri='http://127.0.0.1:5000/authorization')
print(url)

#http://127.0.0.1:5000/authorization?state=&code=0cfa95587f7f320a002dc2eaa89d9b1693aba03a&scope=read,activity:read
#0cfa95587f7f320a002dc2eaa89d9b1693aba03a

http://127.0.0.1:5000/authorization?state=&code=e2e75695129d7dae82394b97917604720476c56d&scope=read,activity:read
e2e75695129d7dae82394b97917604720476c56d



https://www.strava.com/oauth/authorize?client_id=108084&redirect_uri=http%3A%2F%2F127.0.0.1%3A5000%2Fauthorization&approval_prompt=auto&scope=read%2Cactivity%3Aread&response_type=code


In [7]:
AUTH_CODE = 'e2e75695129d7dae82394b97917604720476c56d'
print('auth_code=', AUTH_CODE)

auth_code= e2e75695129d7dae82394b97917604720476c56d


### Exchange auth code for access token + refresh token

In [5]:
client = Client()
token_response = client.exchange_code_for_token(client_id=CLIENT_ID,
                                              client_secret=CLIENT_SECRET,
                                              code=AUTH_CODE)
access_token = token_response['access_token']
refresh_token = token_response['refresh_token']  # You'll need this in 6 hours
print(token_response)

with open('strava.token', 'w') as outfile:
    json.dump(token_response, outfile)


No rates present in response headers


Fault: 400 Client Error: Bad Request [Bad Request: [{'resource': 'AuthorizationCode', 'field': 'code', 'code': 'invalid'}]]

In [10]:
# Retrieve tokens from token file
token_string = ''
with open('strava.token', 'r') as infile:
    token_string = json.load(infile)
print(token_string)

access_token = token_string['access_token']
refresh_token = token_string['refresh_token']
expires_at = token_string['expires_at']

{'access_token': '4036f218c5bfb98132b851bb6358091b5a4be18a', 'refresh_token': '0967598ae6324d33c3790bfe68e7a1cf84f42e20', 'expires_at': 1714391942}


### Refresh access token


In [11]:
token_response = client.refresh_access_token(client_id=CLIENT_ID,
                                      client_secret=CLIENT_SECRET,
                                      refresh_token=refresh_token)
print(token_response)

with open('strava.token', 'w') as outfile:
    json.dump(token_response, outfile)

No rates present in response headers


{'access_token': '65accda439e6fb67195e96c634f2f926996c18b0', 'refresh_token': '0967598ae6324d33c3790bfe68e7a1cf84f42e20', 'expires_at': 1714564552}


### check when token expires

In [7]:
from datetime import datetime

expires_at = '1714564552'
print('expires_at=',expires_at)
print(datetime.fromtimestamp(int(expires_at)))
print(datetime.fromtimestamp(int(expires_at)).strftime("%Y%m%d%H%M%S"))



expires_at= 1714564552
2024-05-01 13:55:52
20240501135552


In [14]:

print(datetime.now())

if datetime.now() < datetime.fromtimestamp(int(expires_at)):
    print('no need to renew the refresh token')
elif datetime.now() > datetime.fromtimestamp(int(expires_at)):
    print('time to renew the refresh token')
else:
    print('dunno')


2024-05-01 07:56:35.938882
no need to renew the refresh token


## Design

Create one schema for historical load of raw data: STG (staging)
Create one schema for transformed model: DM (data mart)
Can I simply create views for DM? --> most likely, yes.

Create log tables
STG_LOG
Purpose: log each ingestion.
Columns: source_id, table_name, load_date, export_date, file_path (incl name), total_rows_in_file, total_rows_ingested
STG_CONFIG
Purpose: describe tables in STG, one record for each table
* which table(s) from each source
* which column is BK in each table

The potential source tables under STG, format <source name>_<api or entity> Eg.
STRAVA_ATHLETE
STRAVA_GEAR
STRAVA_ACTIVITY
REF_DATE
KNMI_weather_something
GARMIN_SLEEP
GARMIN_ATHLETE
GARMIN_period_something

Keep all original column names in STG, even if they contain nulls.

Extra audit columns:
LOAD_DATE
UPDATE_DATE
HASH (all orig columns in alphabetical order)
BK (the ID from Strava, or eventually fake ID from other sources)


Script 1: AUTHENTICATE

Usage


Logic

idea --> put tokens for all sources into one file called all.tokens
example format:
{"source":"strava", "access_token": "xxxxx", "refresh_token": "xxxxx", "expires_at": 20240414112244, "last_load_date": 20240414112244}
{"source":"garmin", "athlete_id": 12345, "last_load_date": 20240414112244}



--------------------------------------------------------

Script 2: RETRIEVE

Usage
export_sport -s <strava> -d <20240414> -c <strava.token>

-s source name, in small letters
-d force date. Retrieve all activities from specified date onwards. If -d is not specified, then it will extract all activities since the last local database load.
-c credentials file containing json in format:
    {"access_token": "xxx", "refresh_token": "xxx", "expires_at": xxx}
    assumes you have run the one-off authenticate script already
    if not specified, assumes token file is in same directory and has the name "<source>.token"

Logic

store parameters in variables
retrieve latest load date from pgres
renew the strava tokens
if source = strava:
    retrieve activities
    make gear list
    retrieve gear

--------------------------------------------------------

Script 3: UPLOAD

Usage
upload_sport -s <strava>

Logic
store parameter(s) in variables
find all file(s) for a given source from input folder (./Raw/<export_date>_<source>_<xxxx>.json)
upload to STG: only load each record if ID doesnt exist yet
move file(s) to processed folder (./Processed/)

File map
./Export/<source>/Raw/
./Export/<source>/Processed/


In [2]:
# Example Hash
import hashlib
import json
data = ['only', 'lists', [1,2,3], 'dictionaries', {'a':0,'b':1}, 'numbers', 47, 'strings']
data_md5 = hashlib.md5(json.dumps(data, sort_keys=True).encode('utf-8')).hexdigest()
print(data_md5)

87e83d90fc0d03f2c05631e2cd68ea02


## Retrieve data using API

### Get Athlete (one-off)

In [15]:
athletes = client.get_athlete()

In [34]:
print("Hello, {}".format(athletes.firstname))

print(athletes)
print(athletes.to_dict())
print('\n')

with open('unit_test\\20240501_athlete.json', 'w') as outfile:
    outfile.write(json.dumps(athletes.to_dict(), indent=4, sort_keys=True, default=str))


Hello, Nerrida
bound_client=<stravalib.client.Client object at 0x000001D941D5D610> id=106384724 city=None country=None created_at=datetime.datetime(2022, 8, 1, 20, 14, 41, tzinfo=datetime.timezone.utc) firstname='Nerrida' lastname='Dempster' premium=False profile='https://dgalywyr863hv.cloudfront.net/pictures/athletes/106384724/25269633/2/large.jpg' profile_medium='https://dgalywyr863hv.cloudfront.net/pictures/athletes/106384724/25269633/2/medium.jpg' resource_state=2 sex=None state=None summit=False updated_at=datetime.datetime(2024, 4, 27, 18, 40, 26, tzinfo=datetime.timezone.utc) bikes=None clubs=None follower_count=None friend_count=None ftp=None measurement_preference=None shoes=None weight=0.0 is_authenticated=None athlete_type=None friend=None follower=None approve_followers=None badge_type_id=0 mutual_friend_count=None date_preference=None email=None super_user=None email_language=None max_heartrate=None username=None description=None instagram_username=None offer_in_app_paymen

### Get Activities (Repeatable)

In [ ]:
activities = client.get_activities(limit=10)

In [44]:
gear_list = set()

with open('unit_test\\20240501_activity.json', 'w') as outfile:
    for activity in activities:
        outfile.write(json.dumps(activity.to_dict(), indent=4, sort_keys=True, default=str))
        gear_list.add(activity.to_dict()['gear_id'])

print(gear_list)


{None, 'g11424741', 'g11260745', 'g17195389'}


### Get Gear
After retrieving all activities, make a list of distinct gear and retrieve it

In [49]:
with open('unit_test\\20240501_gear.json', 'w') as outfile:
    for gear_id in gear_list:
        if gear_id is not None:
            gear = client.get_gear(gear_id)
            outfile.write(json.dumps(gear.to_dict(), indent=4, sort_keys=True, default=str))


## Ingest to database

Run program PGAdmin4

In the left-hand pane, navigate to Databases. Right-click and create a new database: Sport_Activity_Tracker

Utility to convert json to pgres ddl
https://konbert.com/convert/json/to/postgres